### 1) Load necessary Python packages

In [610]:
import os
import sys
import dill

# Add the local src directory to the path
sys.path.append('./src/')

# Load the GBA_model class
from GBA_model import *

### 2) Define a function to load a model from a binary backup
See the script `csv_to_binary.py`.

In [611]:
### Load the model from a binary file ###
def load_model( model_name ):
    filename = "./binary_models/"+model_name+".gba"
    assert os.path.isfile(filename), "ERROR: model not found."
    ifile = open(filename, "rb")
    model = dill.load(ifile)
    ifile.close()
    return model

### 3) Load the model "EC12b" (a very abstract model of <em>E. coli</em>)

In [612]:
# I previously saved the model as a binary file with the script `csv_to_binary.py`
model = load_model("A")

### 4) Define the condition
The condition defines the environmental state of the cell (the composition of the medium). External concentrations are stored in the vector $x$.
Conditions are listed for the EC12b model in `./csv_models/EC12b/conditions.csv`.
We will use the condition 1.

In [613]:
model.set_condition("1")

### 5) Calculate the initial state
This little linear program routine allows us to find at least one suboptimal solution for the GBA model.
The solution (like every solutions) must satisfy the following constraints:
- $c > 0$ (positivity of mass concentrations)
- $p > 0$ (positivity of protein mass concentrations)
- $sM . f = 1$ (Relative density of 1)

The routine requires GUROBIpy to be installed.

In [614]:
model.solve_local_linear_problem()

### 6) Calculate all model variables

In [615]:
model.calculate()

### 7) Display a few variables

In [616]:
print("\n> Summary of the model:")
print(model)


> Summary of the model:

 -------- Model report: A --------
| • Nb metabolites          = 3
| • Nb external metabolites = 1
| • Nb internal metabolites = 2
 ---------------------------------
| • Nb reactions          = 2
| • Nb exchange reactions = 1
| • Nb internal reactions = 0
 ---------------------------------



In [617]:
print("\n> Flux fractions f:")
df = pd.DataFrame(data=model.f, index=model.reaction_ids, columns=["f"])
df


> Flux fractions f:


,f
rxn1,1.00
Ribosome,0.95


In [618]:
print("\n> Fluxes v:")
df = pd.DataFrame(data=model.v, index=model.reaction_ids, columns=["v"])
df


> Fluxes v:


,v
rxn1,784.493843
Ribosome,745.269151


In [619]:
print("\n> Mass fractions b:")
df = pd.DataFrame(data=model.b, index=model.c_ids, columns=["b"])
df


> Mass fractions b:


,b
AA,0.05
Protein,0.95


In [620]:
print("\n> Mass concentrations c:")
df = pd.DataFrame(data=model.c, index=model.c_ids, columns=["c"])
df


> Mass concentrations c:


,c
AA,17.0
Protein,323.0


In [621]:
print("\n> Growth rate mu:")
print(model.mu)


> Growth rate mu:
2.3073348331964825


### 8) Test the consistency of the model (positivity + total density)
The result of the consistency check is stored in the variable model.consistent

In [622]:
model.check_model_consistency()
print(model.consistent)

True


### 9) Mu derivatives
The function `model.calculate()` also calculates derivatives $\dfrac{d\mu}{df}$.
To follow the fitness gradient, the important variable is `GCC_f`, which corrects for total density.

In [623]:
print("\n> dmu/df:")
df = pd.DataFrame(data=model.dmu_f, index=model.reaction_ids, columns=["dmu/df"])
df


> dmu/df:


,dmu/df
rxn1,10.859334
Ribosome,-10.829544


In [624]:
print("\n> GCC_f:")
df = pd.DataFrame(data=model.GCC_f, index=model.reaction_ids, columns=["GCC_f"])
df


> GCC_f:


,GCC_f
rxn1,0.000000
Ribosome,-10.829544


In [625]:
f_truncIndex = ['Ribosome'] #first element index removed
print("\n> f_trunc:")
df = pd.DataFrame(data=model.f_trunc, index=f_truncIndex, columns=["f_trunc"])
df


> f_trunc:


,f_trunc
Ribosome,0.95


In [626]:
def trajectory(model):
  threshold = 1e-5                   # for gradient '=' 0
  consistent_f = np.copy(model.f_trunc) # safes consistent_f
  next_f = np.copy(model.f_trunc)     # the f_trunc, that we are going to change
  allGCC_F = [model.GCC_f[1:]]
  print("first next_f: ")
  print(next_f)
  print("first GCC_f: ")
  print(model.GCC_f)
  t = 0                              # time
  dt = 0.01
  max_time = 1000
  while (t < max_time): # end loop if time is up

    if((np.abs(model.GCC_f) <= threshold ).all()):
      print("absolute is less in GCC_F :", model.GCC_f)
      break
    
    next_f = np.add(next_f, model.GCC_f[1:] * dt)                                      # add without first index of GCC_F
    #print(f"np.add({next_f}, {model.GCC_f} * {dt})")
    model.set_f(next_f)
    model.calculate()                                                           #calculate everything
    #print("-----------------")
    #print(dt)
    #print ("GCC_f ")
    #print(model.GCC_f[1:])
    #print("-----------------")
    #print("next_f: ")
    #print(next_f)
    allGCC_F = np.vstack((allGCC_F,model.GCC_f[1:]))
    model.check_model_consistency()     #check consistency
    if (model.consistent):
      #print("Model is consistent with GCC_F:")
      #print(model.GCC_f)
      t = t + dt                     # calc. new t
      consistent_f = next_f            # saves new f as the consistent f

    else:
      next_f = consistent_f          #resets next_f to last consistent_f
      model.set_f(consistent_f)

      if (dt > 1e-100):             # make sure dt is not too small
       dt = dt * 0.9
       t = t + dt                   # calc. new t
      else:
        break
      

  #print("consistent_f: ")
  #print(consistent_f)
  #print('Last GCC_f')
  #print(model.GCC_f)
  print(allGCC_F)
  
  return model.GCC_f, model.f, model.v

print(trajectory(model))
print(model.consistent)

first next_f: 
[0.95]
first GCC_f: 
[  0.         -10.82954389]
absolute is less in GCC_F : [ 0.00000000e+00 -9.64858268e-06]
[[-1.08295439e+01]
 [-5.12471712e-01]
 [-4.05967233e-01]
 [-3.27179282e-01]
 [-2.66953266e-01]
 [-2.19823193e-01]
 [-1.82292213e-01]
 [-1.52003848e-01]
 [-1.27304463e-01]
 [-1.06995511e-01]
 [-9.01851816e-02]
 [-7.61954072e-02]
 [-6.45012995e-02]
 [-5.46903956e-02]
 [-4.64344559e-02]
 [-3.94694620e-02]
 [-3.35811218e-02]
 [-2.85941620e-02]
 [-2.43642828e-02]
 [-2.07720213e-02]
 [-1.77180052e-02]
 [-1.51192388e-02]
 [-1.29061634e-02]
 [-1.10203094e-02]
 [-9.41240401e-03]
 [-8.04083526e-03]
 [-6.87039564e-03]
 [-5.87124950e-03]
 [-5.01807926e-03]
 [-4.28937645e-03]
 [-3.66685059e-03]
 [-3.13493454e-03]
 [-2.68036953e-03]
 [-2.29185584e-03]
 [-1.95975826e-03]
 [-1.67585720e-03]
 [-1.43313800e-03]
 [-1.22561231e-03]
 [-1.04816650e-03]
 [-8.96432805e-04]
 [-7.66679800e-04]
 [-6.55719103e-04]
 [-5.60825942e-04]
 [-4.79671443e-04]
 [-4.10264884e-04]
 [-3.50904436e-04]


In [627]:
print((np.abs(model.GCC_f[1:]) > 0.1))
print(model.consistent)
#4.42560776e-15

[False]
True
